In [1]:
# scPileupVars = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/needle_post/peaks_chip_genes/CHIP_b1_Input/concat_scPileupVars/pileup.tsv"
# vcf = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/variants.all.vcf"
# outdir = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/needle_post/CHIP_b1_Input/cells_vars/vcfpad_1/"
# vcf_pad = 1


# vcf = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/variants.all.vcf"
# scPileupVars = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/needle_post/peaks_all/CHIP_b1_Input/concat_scPileupVars/pileup.tsv"
# vcf_pad = 1
# outdir = "/data/Mito_Trace/output/somatic_variants/CHIP_dec172021/aggregate/needle_post/peaks_all/CHIP_b1_Input/cells_vars/vcfpad_1"

vcf = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/somatic_variants/regions_atac_peaks/gatk_mutect/variants.vcf.gz"
scPileupVars = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/somatic_variants/regions_atac_peaks/gatk_mutect/post/pileup/Input/concat_scPileupVars/pileup.tsv.gz"
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/somatic_variants/regions_atac_peaks/gatk_mutect/post/pileup/Input/cells_vars/vcfpad_1"
vcf_pad=1

In [2]:
from glob import glob
from os.path import basename, join
from src.utils.data_io import read_csv_multichar
import pandas as pd
from pandarallel import pandarallel
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np

fig_utils from mplh


In [3]:
pandarallel.initialize(nb_workers=16)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
region_f = vcf+".region.txt"
vcf_df = read_csv_multichar(vcf, sep="\t", multicomment="##")
vcf_df[["#CHROM", "POS"]].to_csv(region_f, index=None, sep="\t", header=None)

#### The vcf file is 1-based, while the pileup positions are 0-based since they were from bam (pysam was the program used)
** Need to double check this

In [5]:
vcf_df = vcf_df.loc[vcf_df["#CHROM"].str.contains("^chr")]
vcf_df

vcf_df["posID"] = vcf_df["#CHROM"] + "_" + (vcf_df["POS"].astype(int)-vcf_pad).astype(str)
vcf_df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Control,Flt3l,Input,posID
0,chr1,10146,.,AC,A,.,.,"AS_SB_TABLE=11,14|160,159;DP=345;ECNT=2;MBQ=37...",GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB,"0|1:7,120:0.901:127:3,23:3,46:6,72:0|1:10146_A...","0|1:5,109:0.933:114:2,23:1,42:4,76:0|1:10146_A...","0|1:13,90:0.834:103:3,36:3,21:11,65:0|1:10146_...",chr1_10145
1,chr1,10180,.,T,C,.,.,"AS_SB_TABLE=29,38|148,142;DP=359;ECNT=2;MBQ=20...",GT:AD:AF:DP:F1R2:F2R1:FAD:PGT:PID:PS:SB,"0|1:3,105:0.913:108:2,17:0,43:5,70:0|1:10146_A...","0|1:8,96:0.895:104:2,23:0,33:5,67:0|1:10146_AC...","0|1:56,89:0.522:145:21,31:23,19:46,68:0|1:1014...",chr1_10179
2,chr1,10601,.,G,A,.,.,"AS_SB_TABLE=7,12|0,6;DP=27;ECNT=1;MBQ=20,37;MF...",GT:AD:AF:DP:F1R2:F2R1:FAD:SB,"0/1:8,5:0.500:13:2,2:3,3:5,5:3,5,0,5","0/1:3,1:0.396:4:1,1:1,0:2,1:1,2,0,1","0/1:8,0:0.143:8:4,0:1,0:5,0:3,5,0,0",chr1_10600
3,chr1,16257,.,G,C,.,.,"AS_SB_TABLE=70,65|4,3;DP=143;ECNT=3;MBQ=20,20;...",GT:AD:AF:DP:F1R2:F2R1:FAD:SB,"0/1:27,2:0.090:29:8,0:10,1:19,1:15,12,1,1","0/1:40,3:0.107:43:9,1:14,1:24,2:22,18,2,1","0/1:68,2:0.048:70:23,0:15,1:39,1:33,35,1,1",chr1_16256
4,chr1,16288,.,C,G,.,.,"AS_SB_TABLE=32,37|9,10;DP=93;ECNT=3;MBQ=29,20;...",GT:AD:AF:DP:F1R2:F2R1:FAD:SB,"0/1:16,2:0.115:18:6,1:4,0:14,1:9,7,1,1","0/1:23,7:0.272:30:4,2:9,2:15,5:12,11,4,3","0/1:30,10:0.226:40:8,4:6,0:23,6:11,19,4,6",chr1_16287


In [6]:
# pileup_files = glob(scPileupVars+"/*.bq.tsv")
# len(pileup_files)

## Run using chunks

In [7]:
from src.somvars import pileup_to_af as pa

In [8]:
chunk_size=10000000
batch_no=1

pileup_vars = []
for chunk in pd.read_csv(scPileupVars, compression='gzip', chunksize=chunk_size, sep="\t", dtype=object):
    batch_no+=1
    pileup_vars.append(pa.process_chunk(chunk, vcf_df))
    print(pileup_vars[-1].shape)
    print(batch_no)
    
# pileup_df = pd.read_csv(scPileupVars,sep="\t", index_col=0, compression='gzip', dtype=object)
# pileup_df.head()

(10000000, 6)
(223508, 11)
2
(10000000, 6)
(225471, 11)
3
(10000000, 6)
(221931, 11)
4
(10000000, 6)
(226926, 11)
5
(10000000, 6)
(224889, 11)
6
(10000000, 6)
(228047, 11)
7
(10000000, 6)
(225299, 11)
8
(10000000, 6)
(225205, 11)
9
(10000000, 6)
(226466, 11)
10
(10000000, 6)
(222604, 11)
11
(10000000, 6)
(228807, 11)
12
(10000000, 6)
(226971, 11)
13
(10000000, 6)
(223932, 11)
14
(10000000, 6)
(227455, 11)
15
(10000000, 6)
(224529, 11)
16
(10000000, 6)
(225901, 11)
17
(10000000, 6)
(227913, 11)
18
(10000000, 6)
(225218, 11)
19
(10000000, 6)
(223978, 11)
20
(10000000, 6)
(224507, 11)
21
(10000000, 6)
(225433, 11)
22
(10000000, 6)
(225741, 11)
23
(10000000, 6)
(225008, 11)
24
(10000000, 6)
(224890, 11)
25
(10000000, 6)
(223045, 11)
26
(10000000, 6)
(223484, 11)
27
(10000000, 6)
(225518, 11)
28
(10000000, 6)
(228137, 11)
29
(10000000, 6)
(227724, 11)
30
(10000000, 6)
(227011, 11)
31
(10000000, 6)
(226323, 11)
32
(10000000, 6)
(224102, 11)
33
(10000000, 6)
(224225, 11)
34
(10000000, 6)
(225

In [9]:
# vcf_df["var"] = vcf_df["posID"] + "_" + vcf_df["ALT"]
# vcf_df = vcf_df.set_index("var")
# vcf_df

## Concat chunks and save

In [13]:
pileup_vars_df = pd.concat(pileup_vars)

In [15]:
pileup_vars_df.to_csv(join(outdir, "af.ref.pileup.tsv.gz"), sep="\t", compression='gzip')
pileup_vars_df[pileup_vars_df["isVar"]].to_csv(join(outdir, "af.pileup.tsv.gz"), sep="\t", compression='gzip')


In [ ]:
# ### When concatenating the column names were added too so need to drop those entries and convert the dtypes

# print(pileup_df.shape)
# pileup_df = pileup_df.loc[~((pileup_df["chr"] == "chr") & (pileup_df["nt"] == "nt"))].copy()
# pileup_df.shape

# pileup_df["posID"] = pileup_df["chr"] + "_" + pileup_df["pos"]
# pileup_df = pileup_df.astype({"pos":int, "BQ": float, "count": int})
# pileup_df["BQ"] = pileup_df["BQ"].astype(int)
# pileup_df.dtypes

# ## Generate coverage for each pos

# num_nts_each = pileup_df.groupby(["chr", "pos", "cell"]).size()
# plt.hist(num_nts_each)
# plt.yscale('log')

# ## Extract just the variants

# vcf_df.head()

# pileup_vars_df = pileup_df.loc[pileup_df["posID"].isin(vcf_df["posID"])].copy()
# pileup_vars_df

# pileup_vars_df["coverage"] = pileup_vars_df.groupby(["chr", "pos", "cell"])["count"].transform(lambda x: sum(x))
# pileup_vars_df["AF"] = pileup_vars_df["count"]/pileup_vars_df["coverage"]
# pileup_vars_df.head()

# vcf_pos_df = vcf_df.set_index("posID")
# vcf_pos_df = vcf_pos_df.loc[~(vcf_pos_df.index.duplicated())]
# vcf_pos_df.head()

# pileup_vars_df["ref"] = pileup_vars_df.apply(lambda x: vcf_pos_df.loc[x["posID"], "REF"], axis=1)
# pileup_vars_df.head()

# # print("Multi-allele positions")
# # pileup_vars_df.loc[pileup_vars_df["count"]!=pileup_vars_df["coverage"]].sort_values(["cell", "posID"]).head(20)

# ## Keep the variants not ref

# vcf_df["ID"] = vcf_df.apply(lambda x: f"{x['posID']}_{x['ALT']}", axis=1)
# assert(vcf_df.set_index("ID").index.duplicated().sum()==0)
# vcf_df = vcf_df.set_index("ID")

# pileup_vars_df["ID"] = pileup_vars_df.apply(lambda x: f"{x['posID']}_{x['nt']}", axis=1)
# pileup_vars_df.head()
                                            

# def is_var(entry, vcf_df):
#     if entry["ID"] in vcf_df.index:
#         return True
#     return False 
# #     curr_v = vcf_df.loc[(vcf_df["posID"] == entry["posID"]) & (vcf_df["ALT"] == entry["nt"])]
# #     if len(curr_v)==1:
# #         return True
# #     elif len(curr_v)>1:
# #         raise ValueError(f"more than one variant? {curr_v}")
# #     return False
# pileup_vars_df["isVar"] = pileup_vars_df.parallel_apply(is_var, args=(vcf_df,), axis=1)

# print(f"total bases including ref {pileup_vars_df.shape[0]}")
# print(f"Variants detected {pileup_vars_df['isVar'].sum()}")

# pileup_vars_df[pileup_vars_df["isVar"]]